In this notebook we read in the Master XML file made using NapaTrackMater and create N, Delta times Attribute dimensional vectors. N being the number of tracks present in the chosen region, R, Delta being the chosen time interval (t_minus + t_plus) {t - t_minus, t + t_plus} and Attributes being the morphological and the dynamic properties associated with cells in the tracks. We concatenate the Attribute componenets over the chosen time interval to create a Delta times Attribute dimensional vector and create a pandas dataframe with these vectors for further analysis.

In [ ]:
!pip install napatrackmater
!pip install git+https://github.com/kapoorlab/cellshape-helper
!pip install git+https://github.com/kapoorlab/cellshape-cloud
!pip install git+https://github.com/kapoorlab/cellshape-cluster    

In [ ]:
%gui qt5
from napatrackmater.Trackvector import TrackVector
from pathlib import Path
from ipywidgets import interactive, widgets
from IPython.display import display
import napari 
from tifffile import imread
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
master_xml_path = Path('C:/Users/rando/Downloads/Mari_project/gt/master_gt_star_hyperstack.xml')
spot_csv_path = Path('C:/Users/rando/Downloads/Mari_project/gt/spots.csv')
track_csv_path = Path('C:/Users/rando/Downloads/Mari_project/gt/tracks.csv')
edges_csv_path = Path('C:/Users/rando/Downloads/Mari_project/gt/edges.csv')
image = imread('C:/Users/rando/Downloads/Mari_project/gt/rawk.tif')
show_tracks = False

shape_dynamic_dict = {'Time':0, 'Track ID':1,  'Dividing':2, 'Number_Dividing':3,  'Radius':4, 'Volume':5,'Eccentricity Comp First':6, 'Eccentricity Comp Second':7, 'Surface Area':8, 'Cluster Class':9, 'Speed':10, 'Motion_Angle':11, 'Acceleration':12, 'Distance_Cell_mask':13, 'Radial_Angle':14, 'Cell_Axis_Mask':15}

In [3]:
if show_tracks:
  viewer = napari.Viewer()
else:
    viewer = None
track_vectors = TrackVector(viewer,image,master_xml_path,spot_csv_path, track_csv_path, edges_csv_path, show_tracks = show_tracks)

Reading Master XML
Iterating over spots in frame
Iterating over tracks 547
getting attributes
obtianed spot attributes
obtained track attributes
obtained edge attributes


In [4]:
def track_setter(deltat, deltax, deltay):
    track_vectors.t_minus = deltat[0]
    track_vectors.t_plus = deltat[-1]
    
    track_vectors.x_start = deltax[0]
    track_vectors.x_end = deltax[-1]
    
    track_vectors.y_start = deltay[0]
    track_vectors.y_end = deltay[-1]
    

In [5]:
track_vector_widgets = interactive(track_setter, deltat = widgets.IntRangeSlider(
    value=[track_vectors.tstart, track_vectors.tend],
    min= track_vectors.tstart,
    max=track_vectors.tend,
    step=1,
    description='Delta Time',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d',
), 
    deltax = widgets.IntRangeSlider(
    value=[track_vectors.xmin, track_vectors.xmax],
    min= track_vectors.xmin,
    max=track_vectors.xmax,
    step=1,
    description='Delta X',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d',
), 
    deltay = widgets.IntRangeSlider(
    value=[track_vectors.ymin, track_vectors.ymax],
    min= track_vectors.ymin,
    max=track_vectors.ymax,
    step=1,
    description='Delta Y',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d',
),                               
                                  
                                  )

track_vector_widgets

interactive(children=(IntRangeSlider(value=(0, 265), continuous_update=False, description='Delta Time', max=26…

In [6]:
track_vectors._interactive_function()

returning shape and dynamic vectors as list 618


In [7]:
current_shape_dynamic_vectors = track_vectors.current_shape_dynamic_vectors

In [53]:
global_shape_dynamic_dataframe = []
for i in range(len(current_shape_dynamic_vectors)):
   
   vector_list = list(zip(current_shape_dynamic_vectors[i]))
   data_frame_list = np.transpose(np.asarray([vector_list[i] for i in range(len(vector_list))])[:,0,:]) 
   
   shape_dynamic_dataframe = pd.DataFrame(data_frame_list, columns =['Time', 'Track ID',  'Dividing', 'Number_Dividing',  'Radius', 'Volume', 'Eccentricity Comp First', 'Eccentricity Comp Second', 'Surface Area', 'Cluster Class', 'Speed', 'Motion_Angle', 'Acceleration', 'Distance_Cell_mask', 'Radial_Angle', 'Cell_Axis_Mask'])
   if len(global_shape_dynamic_dataframe) == 0:
        global_shape_dynamic_dataframe = shape_dynamic_dataframe
   else:
        global_shape_dynamic_dataframe = pd.concat([global_shape_dynamic_dataframe, shape_dynamic_dataframe],ignore_index=True)

global_shape_dynamic_dataframe = global_shape_dynamic_dataframe.set_index('Track ID')
global_shape_dynamic_dataframe = global_shape_dynamic_dataframe.sort_values(by=['Track ID'])
global_shape_dynamic_dataframe = global_shape_dynamic_dataframe.sort_values(by=['Time'])


In [54]:
global_shape_dynamic_dataframe.head()

,Time,Dividing,Number_Dividing,Radius,Volume,Eccentricity Comp First,Eccentricity Comp Second,Surface Area,Cluster Class,Speed,Directional_Change_Rate,Acceleration
Track ID,,,,,,,,,,,,
3.0,0.0,0.0,0.0,7.252325,1678.0,2.059771,1.535897,1.898876,0.0,0.0,0.0,0.0
1.0,0.0,0.0,0.0,7.599891,1931.0,1.922295,1.275753,2.087376,0.0,0.0,0.0,0.0
5.0,0.0,0.0,0.0,5.670283,802.0,1.789056,1.634838,1.768206,1.0,0.0,0.0,0.0
4.0,0.0,0.0,0.0,7.185439,1632.0,2.598391,1.810684,1.729526,0.0,0.0,0.0,0.0
0.0,0.0,0.0,0.0,7.144109,1604.0,2.532273,1.923947,0.804023,1.0,0.0,0.0,0.0


In [55]:
global_shape_dynamic_dataframe.tail()

,Time,Dividing,Number_Dividing,Radius,Volume,Eccentricity Comp First,Eccentricity Comp Second,Surface Area,Cluster Class,Speed,Directional_Change_Rate,Acceleration
Track ID,,,,,,,,,,,,
540.0,265.0,0.0,0.0,6.95662,1481.0,3.87495,3.251115,2.335365,2.0,0.008054,149.406584,0.014313
496.0,265.0,1.0,1.0,4.700947,457.0,3.264422,2.919953,0.828997,1.0,0.005257,142.481307,0.008836
499.0,265.0,0.0,0.0,7.186906,1633.0,4.503006,3.255669,2.614231,2.0,0.001542,158.057448,0.004458
542.0,265.0,0.0,0.0,5.639478,789.0,5.409403,2.752201,1.486635,2.0,0.004634,145.370341,0.007037
493.0,265.0,0.0,0.0,7.37277,1763.0,3.338725,3.001985,2.61705,2.0,0.003523,150.328362,0.005708
